<a href="https://colab.research.google.com/github/advapplab/esgBERT_hf/blob/main/NSP_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! wget -nc https://raw.githubusercontent.com/advapplab/esgBERT_hf/main/data/eval/pretraining_testingdata.csv

--2022-12-06 13:41:24--  https://raw.githubusercontent.com/advapplab/esgBERT_hf/main/data/eval/pretraining_testingdata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1808173 (1.7M) [text/plain]
Saving to: ‘pretraining_testingdata.csv’

pretraining_testing 100%[===================>]   1.72M  --.-KB/s    in 0.06s   

2022-12-06 13:41:24 (30.6 MB/s) - ‘pretraining_testingdata.csv’ saved [1808173/1808173]



In [9]:
!pip install transformers datasets pandas numpy evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached datasets-2.7.1-py3-none-any.whl (451 kB)
  Using cached evaluate-0.3.0-py3-none-any.whl (72 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached multiprocess-0.70.14-py38-none-any.whl (132 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [10]:
import requests
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
import datasets
import pandas as pd
input_df = pd.read_csv('pretraining_testingdata.csv', encoding = 'ISO-8859-1')

In [13]:
news_list = input_df['content'].tolist()
news_list

['By the time President Joe Biden articulated it officially Thursday on Earth Day, the new U.S. pledge to halve emissions over the next nine years was one of the worst-kept secrets in climate action circles. Neither was it surprising that the U.S. president closely linked America¡¦s new nationally determined contribution, a reduction target of 50 to 52 percent from a 2005 baseline aligned with the terms of the Paris Agreement, with equitable job creation and the opportunity to make money. "The countries that take decisive action now to create the industries of the future will be the ones that reap the economic benefits of the clean energy boom that¡¦s coming," Biden said in his opening remarks during the virtual two-day climate summit. Attended by more than 40 world leaders, the event last week included both close allies and representatives of countries whose cooperation will be critical to keeping global warming well below 2 degrees Celsius by 2050, including Brazil\'s president, Jair

In [14]:
bag = [sentence for para in news_list for sentence in para.split('.') if sentence != '']
bag_size = len(bag)

In [15]:
import random

sentence_a = []
sentence_b = []
labels = []

for paragraph in news_list:
  sentences = [
      sentence for sentence in paragraph.split('.') if sentence != ''
  ]
  num_sentences = len(sentences)
  if num_sentences > 1:
    start = random.randint(0, num_sentences-2)
    sentence_a.append(sentences[start])
    if random.random() > 0.5:
      sentence_b.append(bag[random.randint(0, bag_size-1)])
      labels.append(1)
    else:
      sentence_b.append(sentences[start+1])
      labels.append(0)


In [16]:
sentence_b[:3]

[' Sadly, there wasn¡¦t nearly enough PPE for relatives to be able to enter the funeral ground or touch the bodies of their dead family members',
 ' However, for a very small number of children at a higher risk of catching the virus and serious illness, the Joint Committee for Vaccination and Immunisation has advised that vaccination can be appropriate, but that this should be a carefully discussed decision between parents/guardians and their GP',
 'Fehrenbacher: Goodyear has been working on replacing petroleum oil with soybean oils']

In [17]:
for i in range(3):
  print(labels[i])
  print(sentence_a[i] + '\n---')
  print(sentence_b[i] + '\n')

1
S
---
 Sadly, there wasn¡¦t nearly enough PPE for relatives to be able to enter the funeral ground or touch the bodies of their dead family members

1
 Do you think the ride-hailing companies need to be more aggressive around electrification? If so, how do you push these companies toward that goal?Pope: We think everyone should be more aggressive around electrification
---
 However, for a very small number of children at a higher risk of catching the virus and serious illness, the Joint Committee for Vaccination and Immunisation has advised that vaccination can be appropriate, but that this should be a carefully discussed decision between parents/guardians and their GP

0
 We¡¦re helping them understand some of the features of those materials that need improvement and then figuring out where they could be applied in our different tires
---
Fehrenbacher: Goodyear has been working on replacing petroleum oil with soybean oils



In [18]:
inputs = tokenizer(sentence_a, sentence_b, max_length=128, truncation=True, padding='max_length', return_tensors="pt")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [19]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [20]:
inputs

{'input_ids': tensor([[ 101, 1055,  102,  ...,    0,    0,    0],
        [ 101, 2079, 2017,  ...,    0,    0,    0],
        [ 101, 2057, 1067,  ...,    0,    0,    0],
        ...,
        [ 101, 1999, 2049,  ...,    0,    0,    0],
        [ 101, 9733, 2623,  ...,    0,    0,    0],
        [ 101, 2023, 2047,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [21]:
inputs['labels'] = torch.LongTensor([labels]).T

In [22]:
eval_dataset = datasets.Dataset.from_dict(inputs)

In [23]:
eval_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 300
})

In [24]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [25]:
inputs.input_ids.shape

torch.Size([300, 128])

In [26]:
from transformers import BertForNextSentencePrediction
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir='/content/')

In [28]:
# outputs = model(**inputs, labels=torch.LongTensor([1]))
# outputs

In [29]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(eval_pred):
#     predictions, targets = eval_pred.predictions, eval_pred.label_ids
#     maxProbInds = np.argmax(predictions, axis=-1)
    
#     mask_pos_preds =  maxProbInds[tuple([targets != -100])]
#     mask_pos_targets = targets[tuple([targets != -100])]
    
#     # python (no numpy)
#     # mask_pos_preds, mask_pos_targets = [], []
#     # seq_count, seq_len = targets.shape
#     # for s in range(seq_count):
#     #     for t in range(seq_len):
#     #         if targets[s][t] != -100:
#     #             mask_pos_preds.append(maxProbInds[s][t])
#     #             mask_pos_targets.append(targets[s][t])
    
#     return metric.compute(predictions=mask_pos_preds, references=mask_pos_targets)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


{'eval_loss': 2.0760698318481445,
 'eval_accuracy': 0.74,
 'eval_runtime': 127.595,
 'eval_samples_per_second': 2.351,
 'eval_steps_per_second': 0.298}